# Dataset Analisys

> to start, let's import all libraries we'll use, as well as our dataset


### Import libraries

In [16]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import plotly.io as pio
import re

# pip install cpi
# da usare 
import cpi


### Import dataset from JSON

In [2]:
JSON_PATH = Path("data/raw.json")

# 1. Open JSON
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. Proceed only if it's a list of records
if isinstance(data, dict):
    data = list(data.values())
elif not isinstance(data, list):
    raise ValueError("The JSON file seems to be empty or not a list of records.")

# 3. Convert to DataFrame, normalizing
df = pd.json_normalize(data)

# 4. Remove empty columns, or columns with all NaN values
df.columns = [col.strip() for col in df.columns]
df = df.loc[:, ~df.columns.str.fullmatch(r"Unnamed:.*|^$")]
df = df.dropna(axis=1, how="all")

### General info about the dataset

In [3]:
print(f"Il dataframe ha {df.shape[1]} colonne.")
print(f"Il dataframe ha {df.shape[0]} righe.")

# Rename some columns
df = df.rename(columns={
    "Power HP": "Power_HP",
})

print(f"La moto più vecchia è del {df['Year'].min()}")
print(f"La moto più recente è del {df['Year'].max()}.")

display(df.columns)

Il dataframe ha 77 colonne.
Il dataframe ha 38624 righe.
La moto più vecchia è del 1894
La moto più recente è del 2021.


Index(['Model', 'Year', 'Category', 'Rating', 'Displacement', 'Engine type',
       'Torque', 'Bore x stroke', 'Fuel system', 'Fuel control',
       'Cooling system', 'Gearbox', 'Transmission type,final drive',
       'Frame type', 'Rake (fork angle)', 'Trail', 'Front suspension',
       'Rear suspension', 'Rear wheel travel', 'Front tyre', 'Rear tyre',
       'Front brakes', 'Diameter', 'Rear brakes', 'Seat height',
       'Ground clearance', 'Wheelbase', 'Fuel capacity', 'Color options',
       'Starter', 'Comments', 'Insurance costs', 'Finance options',
       'Parts finder', 'Ask questions', 'Related bikes', 'Price as new',
       'Engine details', 'Power', 'Compression', 'Valves per cylinder',
       'Ignition', 'Lubrication system', 'Clutch', 'Driveline',
       'Fuel consumption', 'Greenhouse gases', 'Emission details',
       'Exhaust system', 'Front wheel travel', 'Wheels', 'Dry weight',
       'Weight incl. oil, gas, etc', 'Power/weight ratio', 'Overall height',
       'Overa

### Define useful functions

In [4]:
# Convert to numeric
def to_float(txt, unit=None):
    if pd.isna(txt): return None
    m = re.search(r"[-+]?\d*\.?\d+", str(txt))
    if not m: return None
    num = float(m.group())
    if unit=="in_to_mm":   num *= 25.4
    if unit=="lbft_to_Nm": num *= 1.35582
    return num

# We use this function to make the fonts bigger, as well as changing the background color
def set_theme(fig, font_size=20, bg_color="#141415", font_color="white", grid_color="#333333"):
    fig.update_layout(
        paper_bgcolor=bg_color,
        plot_bgcolor=bg_color,
        font=dict(size=font_size, color=font_color),
        xaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
        yaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
        legend=dict(bgcolor="rgba(0,0,0,0)")  # Transparent legend background
    )
    return fig

# Convert Price to numeric
def parse_price(value):
    if pd.isna(value):
        return np.nan
    
    # Search for currency and amount
    # Example: "Euro 9990,00" o "US$ 9990.00"
    match = re.search(r'(Euro|US\$)\s*([\d,\.]+)', str(value))
    if not match:
        return np.nan
    
    currency, amount_str = match.groups()
    
    # Convert to float and handle commas and periods
    # Example: "9990,00" -> 9990.00
    amount = float(amount_str.replace(',', '').replace('.', ''))
    
    # Handle different currencies, we use a fixed conversion rate for simplicity
    if currency == 'Euro':
        return amount * 1.1 
    elif currency == 'US$':
        return amount
    else:
        return np.nan
    
# Extract the numeric part of the Power column
def extract_hp(value):
    match = re.search(r'\d+', str(value))
    return int(match.group()) if match else None
  
# Extract the float part of the rating column
def extract_rating(text):
    if isinstance(text, str):
        match = re.search(r"([0-9]+(?:\.[0-9]+)?)", text)
        if match:
            return float(match.group(1))
    return np.nan

# Cpi gets us the inflation-adjusted price from 1913
def adjust_price_for_inflation(row):
    try:
        price = float(row["Price"])
        year = int(row["Year"])
        return cpi.inflate(price, year, to=2021)
    except:
        return np.nan



def show_col(col):
    # Returns head(5) of a column, but ignores NaNs
    return df[df[col].notna()].sample(5)[col]



### Particular columns to analyze

In [5]:
# Useless columns, to drop

print("\nUSELESS COLUMNS\n\n")

display(show_col("Insurance costs"))

display(show_col("Ask questions"))

display(show_col("Rating"))

display(show_col("Comments"))

display(show_col("Modifications compared to previous model"))

df.drop(columns=["Insurance costs", "Ask questions", "Comments"], inplace=True)



# Interesting columns, perhaps to analyze
print("\nUSEFUL COLUMNS\n\n")

display(show_col("Greenhouse gases"))
# injection or carbuettor
display(show_col("Fuel system"))

display(show_col("Model"))



USELESS COLUMNS




29064    Compare US insurance quotes from the nation's ...
15313    Compare US insurance quotes from the nation's ...
26017    Compare US insurance quotes from the nation's ...
11631    Compare US insurance quotes from the nation's ...
28278    Compare US insurance quotes from the nation's ...
Name: Insurance costs, dtype: object

1872     Join the 20 Husqvarna FC 450 Rockstar Edition ...
38581    Join the 06 NSU 3 1-2 PS discussion group or t...
15131    Join the 11 AJS DD125E Mk2 discussion group or...
3942     Join the 19 Cobra CX50 JR discussion group or ...
9366     Join the 15 AJP PR4 240 Supermoto discussion g...
Name: Ask questions, dtype: object

32930     3.4  See the detailed rating of engine perfor...
3754     Do you know this bike?Click here to rate it. W...
9586     Do you know this bike?Click here to rate it. W...
33086     3.3  Check out the detailed rating of racing ...
27443     3.6  See the detailed rating of design and lo...
Name: Rating, dtype: object

37735                 Also called ´Squariel´.
25777    Model also known as Veli Motor ZQ90.
18591                      Italian made bike.
11025                           Spanish bike.
19262                 Three wheel motorcycle.
Name: Comments, dtype: object

4721     \tNew for 2019, the SV650’s Metallic Oort Gray...
17271    Second oxygen sensor and updated EFI\r\ncalibr...
19548                          New lowprofile front fender
15718                  USHighland also used as brand name.
23289                                                     
Name: Modifications compared to previous model, dtype: object


USEFUL COLUMNS




718       65.0 CO2 g/km. (CO2 - Carbon dioxide emission) 
22053     51.0 CO2 g/km. (CO2 - Carbon dioxide emission) 
32630     65.0 CO2 g/km. (CO2 - Carbon dioxide emission) 
7925     126.9 CO2 g/km. (CO2 - Carbon dioxide emission) 
12518    121.3 CO2 g/km. (CO2 - Carbon dioxide emission) 
Name: Greenhouse gases, dtype: object

18836    Injection. Twin-Bore electronic fuel injection...
37495                                    Carburettor. Amal
27373                                          Carburettor
20666                                          Carburettor
9112                                           Carburettor
Name: Fuel system, dtype: object

38251    Harley-Davidson Eight-valve racer
35008                  Bridgestone 350 GTO
1112                      CF Moto ST Papio
20729                         BMW K 1200 S
16654            Ultra Motor Marathon-Lite
Name: Model, dtype: object

### Cleaning of dataset and inizializations

In [6]:

# Convert certain columns to numeric
df["Displacement_cc"]   = df["Displacement"].apply(to_float)
df["Torque_Nm"]         = df["Torque"].apply(lambda x: to_float(x, unit="lbft_to_Nm") or to_float(x))
df["Seat_height_mm"]    = df["Seat height"].apply(to_float)
df["Fuel_capacity_l"]   = df["Fuel capacity"].apply(to_float)
df["Year"]   = df["Year"].apply(to_float)

# Show only the first word of each model, this is the manufacturer
df["Manufacturer"] = df["Model"].apply(lambda x: re.sub(r"\s.*", "", x) if isinstance(x, str) else x)

df["Price"] = df["Price as new"].apply(parse_price)
df["HP"] = df["Power"].apply(extract_hp)

df["Rating"] = df["Rating"].apply(extract_rating)

# palette & template
TEMPLATE = "presentation"
pio.templates.default = TEMPLATE

# Filter out motorcycle with HP >= 500
df = df[df["HP"] < 500]



# Adjust price for inflation

# Filtra solo righe valide
mask = (df["Year"] >= 1913) & (df["Price"].notna())
df_valid = df.loc[mask].copy()

# Calcola i fattori di inflazione solo una volta per ogni anno
unique_years = df_valid["Year"].unique()
inflation_factors = {
    year: cpi.inflate(1, int(year), to=2021) for year in unique_years
}

# Applica il fattore in modo vettoriale
df_valid["Price_adj"] = df_valid["Price"] * df_valid["Year"].map(inflation_factors)

# Assegna al dataframe originale (solo dove ha senso)
df.loc[mask, "Price_adj"] = df_valid["Price_adj"]

# Show the first 5 rows of the adjusted prices
print("\nADJUSTED PRICES\n\n")
display(df[["Year", "Price", "Price_adj"]].head(5))


ADJUSTED PRICES




,Year,Price,Price_adj
1,2021.0,10989.0,10989.0
2,2021.0,NaN,NaN
3,2021.0,NaN,NaN
5,2021.0,NaN,NaN
6,2021.0,12155.0,12155.0


### Analysis of duplicates


In [7]:
emission_cols = [
    "Emission details",        # ← contiene "Euro 4", "Euro 5", ecc.
    "Greenhouse gases",        # ← valore numerico g/km
    "Fuel consumption"         # ← utile in confronto alle emissioni
]
performance_cols = [
    "Power",                   # può differire se depotenziata
    "Torque",                  # idem
    "Weight incl. oil, gas, etc", # peso a pieno carico
    "Dry weight",              # a secco
    "Power/weight ratio",      # calcolato ma utile per confronto
    "Top speed",               # a volte limitata
]
mechanical_cols = [
    "Engine type",
    "Displacement",
    "Fuel control", 
    "Clutch", 
    "Exhaust system"
]
important_cols = [
    "Manufacturer", "Model", "Category", "Year"
] + emission_cols + performance_cols + mechanical_cols


# Trova i duplicati solo basati su Model, Manufacturer, Category, Year
dups = df.duplicated(subset=["Manufacturer", "Model", "Category", "Year"], keep=False)

# Seleziona solo le righe duplicate
df_dups = df[dups]

# Conta il numero di versioni uniche in base a un sottoinsieme tecnico
df_unique_tech = df_dups.drop_duplicates(subset=important_cols)

print(f"Duplicati totali: {df_dups.shape[0]}")
print(f"Unici in base a info tecniche: {df_unique_tech.shape[0]}")

print(f"Righe originali: {df.shape[0]}")
df = df.drop_duplicates(subset=important_cols, keep="first")

print(f"Righe nuove: {df.shape[0]}")





Duplicati totali: 4615
Unici in base a info tecniche: 2289
Righe originali: 26258
Righe nuove: 23932


## Charts
 > Let's finnally see some charts

### Models by category
 > Let's see the most common type of motorcycle, leaving out those that appear less in the dataset


In [8]:
# Group by most used, leave the others in "Other"
df_grouped = df.groupby("Category").size().reset_index(name="count")


df_top_types = df_grouped[df_grouped["count"] >= 1500].sort_values(by="count", ascending=False)
other_types = df_grouped[df_grouped["count"] < 1500].sum()["count"]

# Create a new DataFrame with the top types and the "Other" category
df_grouped = pd.concat([df_top_types, pd.DataFrame([{'Category': 'Other', 'count': other_types}])], ignore_index=True)

# Concat the top types and the "Other" category
df_pie = pd.concat(
    [df_top_types, pd.DataFrame([{"Category": "Other", "count": other_types}])],
    ignore_index=True
)

# Sort the DataFrame by count in descending order
ordered_categories = list(df_top_types["Category"]) + ["Other"]
df_pie["Category"] = pd.Categorical(
    df_pie["Category"], categories=ordered_categories, ordered=True
)
gray_seq = ["gray"] + ["gray"] + ["gray"] + ["white"]
color_seq = px.colors.qualitative.Plotly[:6] + gray_seq  # bianco per Other


# Create the pie chart
fig1 = px.pie(
    df_pie,
    names="Category",
    values="count",
    title="<b>Models by Type of bike</b>",
    hole=0.35,
    color_discrete_sequence=color_seq,
    category_orders={"Category": ordered_categories},
)

# Rotate the pie chart, make it counterclockwise, don't atutomatically sort
fig1.update_traces(
    direction="counterclockwise",
    sort=False,
    rotation=-61
)


set_theme(fig1).show()

### How many models were created each year
 > Let's see for each category of bike, how many bikes were created

In [9]:
# Seleziona le 6 categorie più presenti
top6 = df_grouped.sort_values("count", ascending=False).head(7)["Category"].tolist()

# Filtra il dataframe originale per le sole 6 categorie
df_top6 = df[df["Category"].isin(top6)]

# Filtra le righe con Year >= 1980
df_top6 = df_top6[df_top6["Year"] >= 1980]
df_top6 = df_top6[df_top6["Year"] < 2021]

# Raggruppa per anno e categoria, conta le moto
year_cat_counts = (
    df_top6.groupby(["Year", "Category"])
    .size()
    .reset_index(name="count")
)

# Grafico a linee
fig = px.line(
    year_cat_counts,
    x="Year",
    y="count",
    color="Category",
    title="<b>Models by Year</b>",
    markers=True,
    color_discrete_sequence=color_seq,
    category_orders={"Category": ordered_categories},
)
set_theme(fig).show()

# 757 000 unità nel 1980 a 217 000 nel 1993
# dal 2020, le normative euro 5 diventano obbligatorie su ogni nuovo modello
#  Per non perdere omologazioni, i costruttori registrano a raffica versioni Euro 4 “final edition” 
#  Ma alcune omologano già anche Euro 5 nello stesso anno, avendo due versioni

dup_cols = ["Manufacturer","Model","Category","Year", "Greenhouse gases"]
dups = df.duplicated(subset=dup_cols, keep=False)


#### What happened in 1993 and '94? Most importantly what happened in 2020, is it something in our dataset? 
 > Thanks to the duplicates analysis we can confidently say that our dataset had an error, but still in 2020 there are lots of new models created

### Counts of bikes per cc
 > Let's figure out how many bikes were produced per cc category

In [10]:
bins   = [0, 125, 400, 700, 1000, 2000, np.inf]
labels = ["0–125 (A1)", "125–400 (A-lim)", "400–700 (A)", 
          "700–1000 (A)", "1000-2000 (A)", "2000+ (A)"]

# A new column with the categories in which every model falls
df["Disp_cat"] = pd.cut(df["Displacement_cc"], bins=bins, labels=labels, right=True)

# For each category, we calculate the count and the mode            
count_df  = (df["Disp_cat"]
             .value_counts()
             .reindex(labels)
             .rename("Count")
             .reset_index()
             .rename(columns={"Disp_cat": "Displacement Range (cc)"}))

# Calulates the mode for each category
mode_df = (
    df.groupby("Disp_cat", observed=False)["Displacement_cc"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    .reindex(labels)
    .reset_index()
    .rename(columns={
        "Disp_cat": "Displacement Range (cc)",
        "Displacement_cc": "mode_cc"
    })
)

# Merge the two DataFrames
disp_summary = count_df.merge(mode_df, on="Displacement Range (cc)")
# Display the mode in a more readable format
disp_summary["mode_text"] = "Mode: " + disp_summary["mode_cc"].round(0).astype(int).astype(str)


fig2 = px.bar(
    disp_summary,
    x="Displacement Range (cc)",
    y="Count",
    #This will show the mode on the bar
    text="mode_text",

    title="<b>Count of Models by CC</b>",
    color_discrete_sequence=["#987434"],
    height=650
)

fig2.update_xaxes(categoryorder="array", categoryarray=labels)

fig2.update_traces(
    texttemplate="%{text}", 
    textposition="inside",
    insidetextanchor="middle", 
    textfont=dict(color="white", size=20)
)



set_theme(fig2).show()


### Price related to a lot of stuff

In [19]:

# Keep only the columns we need for this scatter
plot_df = df.dropna(subset=["Year", "Displacement_cc", "HP", "Price_adj"])

plot_df = plot_df[plot_df["Year"] > 2000]
plot_df = plot_df[plot_df["Displacement_cc"] < 3000]


fig3 = px.scatter(
    plot_df,
    x="Year",
    y="Displacement_cc",
    size="Price_adj",
    color="HP",
    hover_name="Model", # when hovering, show the model, really cute
    color_continuous_scale="YlOrRd",
    range_color=[0, 200],
    opacity=0.6,
    height=700,
    title="Motorcycle Prices by Displacement and Year",
    labels={
        "Year": "Year",
        "Displacement_cc": "Displacement (cc)",
        "Price": "Price (scaled size)",
        "HP": "Horsepower"
    }
)
# Remove the borders from every point
fig3.update_traces(marker=dict(line=dict(width=0)))

# In this for we add the horizontal lines, these are the limits for the A1, A-lim, A and A2 licenses

fig3.update_yaxes(
    tickvals=[125, 250, 600, 1000, 2000],
    title="Displacement (cc)"
)

# Asse x con tick fissi (opzionale)
fig3.update_xaxes(
    tickvals=[1980, 1990, 2000, 2010, 2015, 2020],
    title="Year"
)

fig3.update_layout(
    coloraxis_colorbar=dict(title="Horsepower (HP)"),
)
set_theme(fig3)
fig3.show()


# riportare i prezzi tenendo conto dell'inflazione

# scatter plot su CC e HP, 
# magari scartare il tempo, interessarsi meglio sul prezzo


### Chart divided in two
 > Our professor doesn't like it

In [15]:

# Keep only the columns we need for this scatter
df_year_cc = df.dropna(subset=["Year", "Displacement_cc", "Price_adj"])
df_year_price = df.dropna(subset=["Year", "Price_adj"])

df_year_cc = df_year_cc[df_year_cc["Displacement_cc"] < 3000]
# create a scatterplot with Year on x, Displacement_cc on y to see if there are any trends
fig4 = px.scatter(
    df_year_cc,
    x="Year",
    y="Displacement_cc",
    color="Category",
    hover_name="Model", # when hovering, show the model, really cute
    color_discrete_sequence=px.colors.qualitative.Plotly,
    opacity=0.5,
    size="Price_adj",
    height=700,
    title="Motorcycle Displacement by Year",
    labels={
        "Year": "Year",
        "Displacement_cc": "Displacement (cc)",
        "Category": "Category"
    }
)
fig4.update_traces(marker=dict(line=dict(width=0)))
set_theme(fig4).show()



### Green fuel, electric, how do they compare

In [13]:
df_electric = df.dropna(subset=["Model", "Engine type"])
df_electric = df_electric[df_electric["Engine type"] == "Electric"]

display(df_electric)

,Model,Year,Category,Rating,Displacement,Engine type,Torque,Bore x stroke,Fuel system,Fuel control,...,Rear percentage of weight,Displacement_cc,Torque_Nm,Seat_height_mm,Fuel_capacity_l,Manufacturer,Price,HP,Price_adj,Disp_cat
91,CSC Motorcycles City Slicker,2021.0,Naked bike,2.0,NaN,Electric,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,762.0,NaN,CSC,2495.0,4.0,2495.000000,NaN
97,Cleveland District First Edition,2021.0,Allround,2.0,NaN,Electric,39.3 Nm (4.0 kgf-m or 29.0 ft.lbs),NaN,NaN,NaN,...,NaN,NaN,53.283726,762.0,NaN,Cleveland,8000.0,2.0,8000.000000,NaN
98,Cleveland District Founders Edition,2021.0,Allround,2.0,NaN,Electric,39.3 Nm (4.0 kgf-m or 29.0 ft.lbs),NaN,NaN,NaN,...,NaN,NaN,53.283726,762.0,NaN,Cleveland,15000.0,17.0,15000.000000,NaN
133,Energica EVA Esse Esse 9 Plus RS,2021.0,Sport,2.0,NaN,Electric,200.0 Nm (20.4 kgf-m or 147.5 ft.lbs),NaN,NaN,NaN,...,NaN,NaN,271.164000,810.0,NaN,Energica,NaN,109.0,NaN,NaN
134,Energica Eva Ribelle RS,2021.0,Naked bike,2.0,NaN,Electric,215.0 Nm (21.9 kgf-m or 158.6 ft.lbs) @ 8211 RPM,NaN,NaN,NaN,...,NaN,NaN,291.501300,790.0,NaN,Energica,NaN,143.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21548,MZ Charly,2008.0,Scooter,1.0,NaN,Electric,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,760.0,NaN,MZ,NaN,1.0,NaN,NaN
21804,Sachs Oliver Electro,2008.0,Scooter,3.1,NaN,Electric,NaN,NaN,Carburettor,NaN,...,NaN,NaN,NaN,827.0,14.0,Sachs,NaN,0.0,NaN,NaN
22090,Vectrix Electric 3-Wheeler,2008.0,Scooter,3.0,NaN,Electric,65.0 Nm (6.6 kgf-m or 47.9 ft.lbs),NaN,NaN,NaN,...,NaN,NaN,88.128300,770.0,NaN,Vectrix,15900.0,26.0,20010.975230,NaN
22091,Vectrix Electric Maxi-Scooter,2008.0,Scooter,3.7,NaN,Electric,65.0 Nm (6.6 kgf-m or 47.9 ft.lbs),NaN,NaN,NaN,...,NaN,NaN,88.128300,770.0,NaN,Vectrix,11000.0,26.0,13844.070914,NaN
